In [ ]:
Data Warehousing Fundamentals
1.
CREATE TABLE products (
  product_id INT PRIMARY KEY,
  product_name VARCHAR(100),
  category VARCHAR(50),
  brand VARCHAR(50),
  price DECIMAL(10, 2)
);
CREATE TABLE customers (
  customer_id INT PRIMARY KEY,
  customer_name VARCHAR(100),
  address VARCHAR(200),
  city VARCHAR(50),
  state VARCHAR(50),
  country VARCHAR(50)
);
CREATE TABLE time (
  date_id INT PRIMARY KEY,
  date DATE,
  day INT,
  month INT,
  year INT
);
CREATE TABLE sales (
  sale_id INT PRIMARY KEY,
  product_id INT,
  customer_id INT,
  date_id INT,
  quantity INT,
  amount DECIMAL(10, 2),
  FOREIGN KEY (product_id) REFERENCES products(product_id),
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (date_id) REFERENCES time(date_id)
);


In [ ]:
2.
CREATE TABLE sales (
  sale_id INT PRIMARY KEY,
  product_id INT,
  customer_id INT,
  date_id INT,
  quantity INT,
  amount DECIMAL(10, 2),
  FOREIGN KEY (product_id) REFERENCES products(product_id),
  FOREIGN KEY (customer_id) REFERENCES customers(customer_id),
  FOREIGN KEY (date_id) REFERENCES time(date_id)
);


In [ ]:
3.
SELECT p.product_name, SUM(s.amount) AS total_sales_amount
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.product_name;

SELECT c.customer_name, SUM(s.amount) AS total_sales_amount
FROM sales s
JOIN customers c ON s.customer_id = c.customer_id
GROUP BY c.customer_name;

SELECT t.month, SUM(s.amount) AS total_sales_amount
FROM sales s
JOIN time t ON s.date_id = t.date_id
GROUP BY t.month;

SELECT p.category, SUM(s.amount) AS total_sales_amount
FROM sales s
JOIN products p ON s.product_id = p.product_id
GROUP BY p.category;
SELECT p.product_name, SUM(s.amount) AS total_sales_amount
FROM sales s
JOIN products p ON s.product_id = p.product_id
JOIN time t ON s.date_id = t.date_id
WHERE p.product_name = 'Product X' AND t.date BETWEEN '2023-01-01' AND '2023-12-31'
GROUP BY p.product_name;


In [ ]:
ETL and Data Integration
1.
import csv

def extract_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)
    return data
def transform_data(data):
    transformed_data = []
    for row in data:
        # Apply business rules or calculations
        transformed_row = {
            'column1': row['column1'],
            'column2': row['column2'],
            'new_column': int(row['column3']) * 2,
        }
        transformed_data.append(transformed_row)
    return transformed_data
import psycopg2

def load_data(data):
    connection = psycopg2.connect(
        host='your_host',
        port='your_port',
        user='your_user',
        password='your_password',
        database='your_database'
    )
    cursor = connection.cursor()
    for row in data:
        # Load data into the data warehouse table
        cursor.execute("INSERT INTO your_table (column1, column2, new_column) VALUES (%s, %s, %s)",
                       (row['column1'], row['column2'], row['new_column']))
    connection.commit()
    cursor.close()
    connection.close()
file_path = 'path_to_your_csv_file.csv'
extracted_data = extract_data(file_path)
transformed_data = transform_data(extracted_data)
load_data(transformed_data)


In [ ]:
2.
import csv
import psycopg2

# Extract data from CSV files
def extract_data(file_path):
    data = []
    with open(file_path, 'r') as file:
        reader = csv.DictReader(file)
        for row in reader:
            data.append(row)
    return data

# Transform data by applying business rules
def transform_data(data):
    transformed_data = []
    for row in data:
        transformed_row = {
            'column1': row['column1'],
            'column2': row['column2'],
            'new_column': int(row['column3']) * 2,
        }
        transformed_data.append(transformed_row)
    return transformed_data

# Load data into the PostgreSQL database
def load_data(data):
    connection = psycopg2.connect(
        host='your_host',
        port='your_port',
        user='your_user',
        password='your_password',
        database='your_database'
    )
    cursor = connection.cursor()
    for row in data:
        cursor.execute("INSERT INTO your_table (column1, column2, new_column) VALUES (%s, %s, %s)",
                       (row['column1'], row['column2'], row['new_column']))
    connection.commit()
    cursor.close()
    connection.close()

# Perform the ETL process
file_path = 'path_to_your_csv_file.csv'
extracted_data = extract_data(file_path)
transformed_data = transform_data(extracted_data)
load_data(transformed_data)


In [ ]:
Dimensional Modeling and Schemas
1.
CREATE TABLE student_enrollments (
  enrollment_id SERIAL PRIMARY KEY,
  student_id INT REFERENCES students(student_id),
  course_id INT REFERENCES courses(course_id),
  time_id INT REFERENCES time(time_id),
  enrollment_date DATE,
  enrollment_status VARCHAR(50),
  grade FLOAT
);
CREATE TABLE students (
  student_id SERIAL PRIMARY KEY,
  student_name VARCHAR(100),
  date_of_birth DATE,
  gender VARCHAR(10),
  major VARCHAR(50)
);

CREATE TABLE courses (
  course_id SERIAL PRIMARY KEY,
  course_name VARCHAR(100),
  department VARCHAR(50),
  credit_hours INT
);

CREATE TABLE time (
  time_id SERIAL PRIMARY KEY,
  date DATE,
  day_of_week VARCHAR(20),
  semester VARCHAR(20),
  year INT
);


In [ ]:
2)
SELECT s.student_name, c.course_name, se.enrollment_date
FROM student_enrollments se
JOIN students s ON se.student_id = s.student_id
JOIN courses c ON se.course_id = c.course_id;
SELECT c.course_name, COUNT(*) AS total_students
FROM student_enrollments se
JOIN courses c ON se.course_id = c.course_id
GROUP BY c.course_name;
SELECT c.course_name, AVG(se.grade) AS average_grade
FROM student_enrollments se
JOIN courses c ON se.course_id = c.course_id
JOIN time t ON se.time_id = t.time_id
WHERE t.semester = 'Fall' AND t.year = 2022
GROUP BY c.course_name;
SELECT c.course_name, s.gender, COUNT(*) AS total_students
FROM student_enrollments se
JOIN students s ON se.student_id = s.student_id
JOIN courses c ON se.course_id = c.course_id
GROUP BY c.course_name, s.gender;
SELECT t.day_of_week, COUNT(*) AS enrollment_count
FROM student_enrollments se
JOIN time t ON se.time_id = t.time_id
GROUP BY t.day_of_week;


In [ ]:
 Performance Optimization and Querying
 1.
 import psycopg2

# Assuming you have a list of data to be inserted
data = [(1, 'John'), (2, 'Jane'), (3, 'Mark')]

# Establish a connection to the database
connection = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

# Create a cursor object
cursor = connection.cursor()

# Define the SQL query
query = "INSERT INTO your_table (id, name) VALUES (%s, %s)"

# Batch insert
cursor.executemany(query, data)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()
import psycopg2

# Assuming you have a list of data to be inserted
data = [(1, 'John'), (2, 'Jane'), (3, 'Mark')]

# Establish a connection to the database
connection = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

# Create a cursor object
cursor = connection.cursor()

# Prepare the SQL statement
query = "INSERT INTO your_table (id, name) VALUES (%s, %s)"
cursor.prepare(query)

# Execute the prepared statement multiple times
for record in data:
    cursor.execute(query, record)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()
import psycopg2

# Assuming you have a CSV file with your data
csv_file = '/path/to/your/file.csv'

# Establish a connection to the database
connection = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

# Create a cursor object
cursor = connection.cursor()

# Use the COPY command to load data from the CSV file
query = f"COPY your_table FROM '{csv_file}' DELIMITER ',' CSV HEADER"
cursor.execute(query)

# Commit the changes
connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()



In [ ]:
a)
import psycopg2

# Assuming you have a list of data to be inserted
data = [(1, 'John'), (2, 'Jane'), (3, 'Mark')]

# Batch size (number of records per batch)
batch_size = 1000

# Establish a connection to the database
connection = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

# Create a cursor object
cursor = connection.cursor()

# Define the SQL query
query = "INSERT INTO your_table (id, name) VALUES (%s, %s)"

# Batch processing
for i in range(0, len(data), batch_size):
    batch_data = data[i:i + batch_size]  # Extract a batch of data

    # Batch insert
    cursor.executemany(query, batch_data)

    # Commit the batch
    connection.commit()

# Close the cursor and connection
cursor.close()
connection.close()


In [ ]:
b)
import psycopg2
from multiprocessing import Pool

# Assuming you have a list of data to be inserted
data = [(1, 'John'), (2, 'Jane'), (3, 'Mark')]

# Number of processes to spawn
num_processes = 4

# Establish a connection to the database
connection = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

# Define the SQL query
query = "INSERT INTO your_table (id, name) VALUES (%s, %s)"

# Function to perform the batch insert
def batch_insert(records):
    # Create a cursor object for each process
    cursor = connection.cursor()

    # Batch insert
    cursor.executemany(query, records)

    # Commit the changes
    connection.commit()

    # Close the cursor
    cursor.close()

# Split the data into chunks for each process
chunk_size = len(data) // num_processes
chunks = [data[i:i + chunk_size] for i in range(0, len(data), chunk_size)]

# Create a pool of processes
pool = Pool(processes=num_processes)

# Map the batch_insert function to the pool of processes
pool.map(batch_insert, chunks)

# Close the connection
connection.close()


In [ ]:
c)
import time
import psycopg2

# Assuming you have a list of data to be inserted
data = [(1, 'John'), (2, 'Jane'), (3, 'Mark')]

# Establish a connection to the database
connection = psycopg2.connect(database="your_database", user="your_user", password="your_password", host="your_host", port="your_port")

# Create a cursor object
cursor = connection.cursor()

# Define the SQL query
query = "INSERT INTO your_table (id, name) VALUES (%s, %s)"

# Measure the execution time before data loading
start_time = time.time()

# Data loading process
cursor.executemany(query, data)

# Commit the changes
connection.commit()

# Measure the execution time after data loading
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

# Close the cursor and connection
cursor.close()
connection.close()

# Print the elapsed time
print(f"Elapsed Time: {elapsed_time} seconds")
